In [ ]:
# First make sure to install all required packages.
# You can do it by running the following command:

In [ ]:
# ]add Arrow CSV DataFrames Plots FreqTables StatsBase

In [ ]:
# If you launched Jupyter in directory with attached Project.toml and Manifest.toml
# use below command to install required packages with fixed versions. 
# Check Project introduction for more information.

In [ ]:
#] instantiate

In [ ]:
# Import required libraries
import Downloads
import SHA
using Arrow
using CSV
using DataFrames
using Plots
using FreqTables
using Statistics
using StatsBase

# Boston Housing

In [ ]:
# Define URL to Boston Housing data and expected SHA1
const HOUSING_URL = "https://archive.ics.uci.edu/ml/" *
                    "machine-learning-databases/housing/housing.data"
const HOUSING_NAME = "housing.txt"
const HOUSING_SHA1 = [0xad, 0xfa, 0x6b, 0x6d, 0xca,
                      0x24, 0xa6, 0x3f, 0xe1, 0x66,
                      0xa9, 0xe7, 0xfa, 0x01, 0xce,
                      0xe4, 0x33, 0x58, 0x57, 0xd1];

In [ ]:
# Download Boston Housing data if not exists
if isfile(HOUSING_NAME)
    @info "$HOUSING_NAME found. Skipping download."
else
    @info "$HOUSING_NAME not found. Fetching from source."
    Downloads.download(HOUSING_URL, HOUSING_NAME)
end

In [ ]:
# Check SHA1 of Boston Housing file
if HOUSING_SHA1 == open(SHA.sha1, HOUSING_NAME)
    @info "SHA1 check of $HOUSING_NAME passed."
else
    error("$HOUSING_NAME file has an invalid SHA1. Aborting!")
end

In [ ]:
# Read Boston Housing CSV
housing_ref = CSV.read(HOUSING_NAME, DataFrame,
                       header=[:CRIM, :ZN, :INDUS, :CHAS, :NOX, :RM, :AGE,
                               :DIS, :RAD, :TAX, :PTRATIO, :B, :LSTAT, :MEDV],
                       delim=' ', ignorerepeated=true, tasks=1)

# We make a copy as we will modify housing variable later
# However, we want to keep housing_ref untouched in case we make some error to avoid reloading it repeatedly
housing = copy(housing_ref)

In [ ]:
# Check basic statistics of all columns
describe(housing)

In [ ]:
# Find nominal variables (output of describe(housing) shows us that integer columns contin only a few values)
nominal = names(housing, Int)

In [ ]:
# Find continuous variables
continuous = names(housing, Float64)

In [ ]:
# Inspect distribution of nominal variables
foreach(name -> println("\n", proptable(housing, name)), nominal)

In [ ]:
# Check distributions of numeric features

# First define a helper function for drawing of a single histogram
histogram_helper(column_name) =
    histogram(housing[!, column_name], xlabel=column_name, legend=false)

# Compose a grid of histograms in a single plot
plot(map(x -> histogram_helper(x), continuous)..., layout=grid(3, 4), size=(800,500))

In [ ]:
# Check the frequency table of :MEDV variable
freqtable(housing, :MEDV)

In [ ]:
# Based on histogram plot, MEDV is censored or inaccurate on upper limit - 16 observations have value equal to 50.0
# In our project we decide to just remove these values

# Remove rows from housing in-place (! marks in-place operation)
# Note that 16 rows got removed
filter!(:MEDV => <(50.0), housing)

In [ ]:
#Check variables distributions after filtering the observations
plot(map(x -> histogram_helper(x), continuous)..., layout=grid(3, 4), size=(800,500))

In [ ]:
#Calculate Kendall's correlation
housing_cor = corkendall(Matrix(housing))

In [ ]:
# Get the information how we should reorder rows of housing_cor
# Remember that MEDV is the last variable in our data set
ord = sortperm(housing_cor[:, end])

# Plot a heatmap, where both axis labels and correlation matrix are reordered by correlation with MEDV
heatmap(names(housing)[ord],
        names(housing)[ord],
        housing_cor[ord, ord],
        c=:balance,
        size=(800,500))

In [ ]:
# Get information on absolute value of correlation
sort(DataFrame(variable = names(housing), cor=housing_cor[:, end]),
     :cor, by=abs)

In [ ]:
#Additionally check relation of continuous variables visually on scatterplots

scatter_helper(column_name) =
    scatter(housing[!, column_name], housing.MEDV, xlabel=column_name,
            legend=false, smooth=true, ms=1)

plot(map(x -> scatter_helper(x), continuous)..., layout=grid(3, 4), size=(800,500))

In [ ]:
# Remove least correlated feature - B
# Again, we do an in-place operation
select!(housing, Not(:B))

In [ ]:
# Transform CRIM and DIS - logarithmic transformation
# Also bin ZN variable
transform!(housing,
           :CRIM => ByRow(log), :DIS => ByRow(log), :ZN => ByRow(>(0)),
           renamecols=false)

In [ ]:
# Recalculate the list of continuous variables
continuous = names(housing, Float64)

In [ ]:
# Plot histogram again
# We see that now distributions of variables look better
plot(map(x -> histogram_helper(x), continuous)..., layout=grid(2, 5), size=(800,500))

In [ ]:
# Plot scatterplot again
plot(map(x -> scatter_helper(x), continuous)..., layout=grid(2, 5), size=(900,500))

In [ ]:
# Declare auxilary function for calculating bootstrap 90% confidence interval
function gen_meanCI(x)
    boot = [mean(rand(x, length(x))) for _ in 1:10_000]
    return (mean=mean(x), q5=quantile(boot, 0.05), q95=quantile(boot, 0.95))
end

In [ ]:
# Mean and 90% CI ends per group for :CHAS variable
mean_chas = combine(groupby(housing, :CHAS, sort=true), :MEDV => gen_meanCI => AsTable)

In [ ]:
# Mean and 90% CI ends per group for :RAD variable
mean_rad = combine(groupby(housing, :RAD, sort=true), :MEDV => gen_meanCI => AsTable)

In [ ]:
# Mean and 90% CI ends per group for :ZN variable
mean_zn = combine(groupby(housing, :ZN, sort=true), :MEDV => gen_meanCI => AsTable)

In [ ]:
# Plot the data frames in a single plot
# You could use @df macro from StatsPlots.jl package to save you some typing here
plot(plot(mean_chas.CHAS, mean_chas.mean,
          yerror=(mean_chas.mean - mean_chas.q5, mean_chas.q95 - mean_chas.mean),
          label=nothing, title="CHAS", seriestype=:scatter),
     plot(mean_rad.RAD, mean_rad.mean,
          yerror=(mean_rad.mean - mean_rad.q5, mean_rad.q95 - mean_rad.mean),
          label=nothing, title="RAD", seriestype=:scatter),
     plot(mean_zn.ZN, mean_zn.mean,
          yerror=(mean_zn.mean - mean_zn.q5, mean_zn.q95 - mean_zn.mean),
          label=nothing, title="ZN", seriestype=:scatter),
    layout=grid(1, 3), size = (800,400))

In [ ]:
# Remove variable RAD as we do not see its interpretable relationship with :MEDV target variable
select!(housing, Not(:RAD))

# Have a look at the data after cleaning
describe(housing)

In [ ]:
# Save clean dataset as Arrow file
Arrow.write("housing.arrow", housing)